# 6 - Poisson equation

Topics covered in this tutorial:

- instantiate [Derham](https://struphy.pages.mpcdf.de/struphy/sections/subsections/feec_derham.html#module-struphy.feec.psydac_derham) objects for different dimensions
- creating callable FE fields
- use of [Projections](https://struphy.pages.mpcdf.de/struphy/sections/subsections/feec_projectors.html#module-struphy.feec.projectors) into Derham
- instantiate [WeigthedMassOperators](https://struphy.pages.mpcdf.de/struphy/sections/subsections/feec_weightedmass.html#module-struphy.feec.mass) object
- use of [Poisson](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_fields.html#struphy.propagators.propagators_fields.Poisson) propagator

In what follows we present some examples of the following problem:
let $\Omega \subset \mathbb R^d$ be open. We want to find $\phi \in H^1(\Omega)$ such that

$$
- \nabla \cdot  \,\nabla \phi(\mathbf x) + \epsilon \,\phi(\mathbf x)  = \rho(\mathbf x)\qquad \mathbf x \in \Omega\,,
$$

for suitable boundary conditions, where $\epsilon \in \mathbb R$ is a constant and $\rho: \Omega \to \mathbb R^+$ is a positive function.

For this we can use the Propagator [Poisson](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_fields.html#struphy.propagators.propagators_fields.Poisson).

In [ ]:
from struphy.propagators.propagators_fields import Poisson

# default parameters of the Propagator
opts = Poisson.options(default=True)
opts

## Manufactured solution in 1D

In [ ]:
# set up Derham complex
from struphy.feec.psydac_derham import Derham

Nel = [32, 1, 1]
p = [1, 1, 1]
spl_kind = [True, True, True]
derham = Derham(Nel, p, spl_kind)

In [ ]:
# set up domain Omega
from struphy.geometry.domains import Cuboid
import numpy as np

l1 = -2 * np.pi
r1 = 2 * np.pi
domain = Cuboid(l1=l1, r1=r1)

In [ ]:
# set up mass matrices
from struphy.feec.mass import WeightedMassOperators

mass_ops = WeightedMassOperators(derham, domain)

In [ ]:
# pass simulation parameters to Propagator
Poisson.derham = derham
Poisson.domain = domain
Poisson.mass_ops = mass_ops

In [ ]:
# create solution field in Vh_0 subset H1
phi = derham.create_spline_function("my solution", "H1")
phi

In [ ]:
phi.vector

In [ ]:
# manufactured solution, defined on Omega
k = 2
f_xyz = lambda x, y, z: np.sin(k * x)
rhs_xyz = lambda x, y, z: k**2 * np.sin(k * x)

# pullback to the logical unit cube
rhs = lambda e1, e2, e3: domain.pull(rhs_xyz, e1, e2, e3)

In [ ]:
# compute rhs vector in Vh_0 subset H1
from struphy.feec.projectors import L2Projector

l2proj = L2Projector("H1", mass_ops)

rho = l2proj.get_dofs(rhs)

In [ ]:
# equation parameters
eps = 1e-12

# instantiate Propagator for the above quation, pass data structure (vector) of FemField
poisson = Poisson(phi.vector, stab_eps=eps, rho=rho)

In [ ]:
# solve (call with arbitrary dt)
poisson(1.0)

In [ ]:
# evalaute at logical coordinates
e1 = np.linspace(0, 1, 100)
e2 = 0.5
e3 = 0.5

funval = phi(e1, e2, e3)

In [ ]:
# push to Omega
fh_xyz = domain.push(funval, e1, e2, e3, squeeze_out=True)

x, y, z = domain(e1, e2, e3, squeeze_out=True)
x.shape

In [ ]:
# plot solution
from matplotlib import pyplot as plt

plt.plot(x, f_xyz(x, 0.0, 0.0), label="exact")
plt.plot(x, fh_xyz, "--r", label="numeric")
plt.xlabel("x")
plt.legend();

## Manufactured solution in a Torus

Under construction ...